In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree
 

In [2]:
from google.colab import files
uploaded = files.upload()


Saving OnlineRetail.csv to OnlineRetail.csv


In [3]:
retail = pd.read_csv(r'OnlineRetail.csv', sep = ",", encoding="ISO-8859-1", header=0)
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [4]:
retail.shape

(541909, 8)

In [5]:
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
retail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


#### Calculating the missing values % in the data

In [7]:
df_null = round(100 * (retail.isnull().sum())/len(retail), 2)
df_null

InvoiceNo       0.00
StockCode       0.00
Description     0.27
Quantity        0.00
InvoiceDate     0.00
UnitPrice       0.00
CustomerID     24.93
Country         0.00
dtype: float64

#### Dropping rows having missing values

In [8]:
retail = retail.dropna()
retail.shape

(406829, 8)

In [9]:
# Changing the datatype of Customer ID as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

## Data Preparation

#### We are going to analysis Customers based on below 3 factors
#### 1) R (Recency) - Number of days since last purchase
#### 2) F (Frequency) - Number of Transactions
#### 3) M (Monetory) - Total amount of transactions (revenue contributed)

In [10]:
retail['Amount'] = retail['Quantity']*retail['UnitPrice']
rfm_m = retail.groupby('CustomerID')['Amount'].sum()
rfm_m = rfm_m.reset_index()
rfm_m.head()

,CustomerID,Amount
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40


In [11]:
rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()
rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']
rfm_f.head()

,CustomerID,Frequency
0,12346.0,2
1,12347.0,182
2,12348.0,31
3,12349.0,73
4,12350.0,17


In [12]:
rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
rfm.head()

,CustomerID,Amount,Frequency
0,12346.0,0.00,2
1,12347.0,4310.00,182
2,12348.0,1797.24,31
3,12349.0,1757.55,73
4,12350.0,334.40,17


#### New Attribute : Recency
#### Convert to date time to proper datatype

In [32]:
retail['InvoiceNo'] = pd.to_datetime(retail['InvoiceDate'], format='%d-%m-%Y %H:%M')


In [34]:
max_date = max(retail["InvoiceDate"])
max_date

'31-10-2011 17:13'

In [38]:
from datetime import datetime
date_str = '31-10-2011 17:13'
date = datetime.strptime(date_str, '%d-%m-%Y %H:%M')
date

datetime.datetime(2011, 10, 31, 17, 13)

In [39]:
from datetime import datetime
date_str_1 = '01-12-2010 08:26'
date_1 = datetime.strptime(date_str_1, '%d-%m-%Y %H:%M')
date_1

datetime.datetime(2010, 12, 1, 8, 26)

In [45]:
# Compute the difference between max date and transaction date

retail['Diff'] = float(date_str_1) - float(date_str)
retail.head()


ValueError: ignored

#### Compute the last transaction date to get the recency of the customers

In [18]:
rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head()

KeyError: ignored